In [2]:
import os
import h5py
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
data_dir='Outputs/'

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
   sys.path.append(module_path)


# make sure we're dealing with the most recent version of any code we're using
%load_ext autoreload
%autoreload 2

In [34]:
ATL03_file=glob('/home/jovyan/crackup/Outputs_New/Edgeworth*ATL03*.h5')
ATL06_file=glob('/home/jovyan/crackup/Outputs_New/Edgeworth*ATL06*.h5')

f = h5py.File(ATL03_file[0], 'r')  # keep it open

# Inspect base groups quickly
# for group in f.keys() :
#     print(group)
#     if group.startswith('gt1l'):
#         for gg in f[group].keys():
#             if gg.startswith('heights'):
#                 for dset in f[group][gg].keys():
#                     print(dset)

# ~code from Ben Smith
beam = [k for k in f.keys() if k.startswith('gt')]
beam = beam[1]
print('beam3',beam)
data = {}
data[beam] = {}
data[beam]['heights'] = {}


for key,val in f[beam]['heights'].items():
    data[beam]['heights'][key] = val[:]

print(data[beam]['heights'].keys())

#-- 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
conf = data[beam]['heights']['signal_conf_ph'][:,3]
lat_ph = data[beam]['heights']['lat_ph']
f.close()

f6 = h5py.File(ATL06_file[0], 'r')
beam6 = [k for k in f6.keys() if k.startswith('gt')]
print(beam6)
beam6 = beam6[1] #1,3,5 strong vs 0,2,4 weak
print('beam6',beam6)
data6 = {}
data6[beam6] = {}
data6[beam6]['land_ice_segments'] = {}

print('what',f6[beam6]['land_ice_segments'].keys())
lookfor = ['delta_time','h_li','h_li_sigma','latitude','longitude','segment_id','sigma_geo_h']
for key,val in f6[beam6]['land_ice_segments'].items():
    if key in lookfor:
        data6[beam6]['land_ice_segments'][key] = val[:]
    #failed attempt at the 'pythonic' way of filtering
#         fv = val.fill_value 
#         np.ma.array(val[:],mask=(val[:]==fv),fill_value=fv)
        
        
print(data6[beam6]['land_ice_segments'].keys())
f6.close()

#identify bad values to filter-out
bv, = np.nonzero(data6[beam]['land_ice_segments']['h_li'] < 5000) #replace 5000 w/ real no-data value

%matplotlib widget
fig, ax = plt.subplots()
# ax.plot(data[beam]['heights']['lat_ph'][conf==1],data[beam]['heights']['h_ph'][conf==1],'.',color='darkgreen')
ax.plot(data[beam]['heights']['lat_ph'][conf==2],data[beam]['heights']['h_ph'][conf==2],'.',color='mediumseagreen')
ax.plot(data[beam]['heights']['lat_ph'][conf==3],data[beam]['heights']['h_ph'][conf==3],'.',color='mediumaquamarine')
ax.plot(data[beam]['heights']['lat_ph'][conf==4],data[beam]['heights']['h_ph'][conf==4],'.',color='lawngreen')
ax.plot(data6[beam]['land_ice_segments']['latitude'][bv],data6[beam]['land_ice_segments']['h_li'][bv],'k-')
ax.set_ylim(0,60)
ax.set_xlim(-64.41,-64.36)
# ax.set_ylim(10,25)
# ax.set_xlim(-71.6,-71.4)
plt.show()

beam3 gt1r
dict_keys(['delta_time', 'dist_ph_across', 'dist_ph_along', 'h_ph', 'lat_ph', 'lon_ph', 'pce_mframe_cnt', 'ph_id_channel', 'ph_id_count', 'ph_id_pulse', 'signal_conf_ph'])
['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
beam6 gt1r
what <KeysViewHDF5 ['atl06_quality_summary', 'bias_correction', 'delta_time', 'dem', 'fit_statistics', 'geophysical', 'ground_track', 'h_li', 'h_li_sigma', 'latitude', 'longitude', 'segment_id', 'sigma_geo_h']>
dict_keys(['delta_time', 'h_li', 'h_li_sigma', 'latitude', 'longitude', 'segment_id', 'sigma_geo_h'])


FigureCanvasNbAgg()

In [35]:
#try to find the nearest neighbors
import numpy as np
import math
import pyproj

#from Fernando Paolo & Johan Nilsson's utilities
def transform_coord(proj1, proj2, x, y):
    """Transform coordinates from proj1 to proj2 (EPSG num)."""
    
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+proj1)
    proj2 = pyproj.Proj("+init=EPSG:"+proj2)
    
    # Convert coordinates
    return pyproj.transform(proj1, proj2, x, y)

def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
#         return array[idx-1]
        return idx-1
    else:
#         return array[idx]
        return idx

#ATL03 data
conf = data[beam]['heights']['signal_conf_ph'][:,3]
photconf = conf[conf>=2]
photlat = data[beam]['heights']['lat_ph'][conf>=2]
photlon = data[beam]['heights']['lon_ph'][conf>=2]
photx, photy = transform_coord('4326','3031', photlon, photlat) #convert to Antarctic PS
photh = data[beam]['heights']['h_ph'][conf>=2]

#ATL06 data
pointlat = data6[beam]['land_ice_segments']['latitude'][bv]
pointlon = data6[beam]['land_ice_segments']['longitude'][bv]
pointx, pointy = transform_coord('4326','3031', pointlon, pointlat) #convert to Antarctic PS
pointh = data6[beam]['land_ice_segments']['h_li'][bv]


# print(find_nearest(pointlat,photlat[200]))
elev_diff = np.zeros_like(photy)
for i,reflat in enumerate(photy):
    dist_array = np.sqrt((pointx-photx[i])**2 + (pointy-reflat)**2)
    idx = np.where(dist_array == dist_array.min())
#     idx = find_nearest(pointlat,reflat)
#     if np.abs(pointlat[idx]-photlat[i]) <= 20:
    if dist_array[idx] <= 20:
        elev_diff[i] = pointh[idx] - photh[i]
    else:
        elev_diff[i] = np.NaN
        photlat[i] = 0
    
    
# print(np.sum(elev_diff))
%matplotlib widget
fig, ax = plt.subplots(2,1)
# ax[0].plot(data[beam]['heights']['lat_ph'][conf>=2],data[beam]['heights']['h_ph'][conf>=2],'.',color='mediumseagreen')
ax[0].plot(photlat[photconf==2],photh[photconf==2],'.',color='mediumseagreen')
ax[0].plot(photlat[photconf==3],photh[photconf==3],'.',color='mediumaquamarine')
ax[0].plot(photlat[photconf==4],photh[photconf==4],'.',color='lawngreen')
ax[0].plot(pointlat,pointh,'k-')
ax[0].set_ylim(0,800)
ax[0].set_xlim(photlat[np.abs(photlat)>0].min(),photlat[np.abs(photlat)>0].max())
ax[1].plot(photlat[photconf==2],elev_diff[photconf==2],'.',color='mediumseagreen')
ax[1].plot(photlat[photconf==3],elev_diff[photconf==3],'.',color='mediumaquamarine')
ax[1].plot(photlat[photconf==4],elev_diff[photconf==4],'.',color='lawngreen')
ax[1].set_ylim(-20,20)
ax[1].set_xlim(photlat[np.abs(photlat)>0].min(),photlat[np.abs(photlat)>0].max())
plt.show()
    

FigureCanvasNbAgg()